# MORAL FOUNDATIONS DICTIONARY TRANSLATION TO SPANISH

In [1]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
import deepl
import pickle
import re
from copy import deepcopy
import pandas as pd
from nltk.stem import SnowballStemmer
# --- 
pd.set_option('display.max_rows', 205)
pd.options.display.max_colwidth = 350

In this notebook we will take the moral foundations dictionary implemented in quanteda, transform it into a Python format, and query an API to translate all words in the dictionary from English to Spanish.

## 1 - Data load and transformation of the dictionary from R:

In [2]:
pandas2ri.activate()

# Credits on reading an RDS file in Python to:
# https://stackoverflow.com/a/42841016/15459665
readRDS = robjects.r['readRDS']
r_dict = readRDS('../data/mfd_dict.rds')

# Translate the MFD dictionary imported from R into a Python dictionary object
mfd_dict = {r_dict.names[i]: list(r_dict[i]) for i in range(len(r_dict.names))}

In [4]:
print(mfd_dict)

{'care.virtue': ['alleviate', 'alleviated', 'alleviates', 'alleviating', 'alleviation', 'altruism', 'altruist', 'beneficence', 'beneficiary', 'benefit', 'benefits', 'benefitted', 'benefitting', 'benevolence', 'benevolent', 'care', 'cared', 'caregiver', 'cares', 'caring', 'caringly', 'charitable', 'charity', 'child', 'childbearing', 'childbirth', 'childcare', 'childhood', 'clothe', 'comfort', 'comforted', 'comforting', 'compassion', 'compassionate', 'compassionately', 'condolence', 'condolences', 'console', 'consoled', 'consoles', 'consoling', 'consolingly', 'empathised', 'empathises', 'empathising', 'empathize', 'empathized', 'empathizers', 'empathizes', 'empathizing', 'empathy', 'feed', 'feeder', 'feeding', 'feeds', 'generosity', 'generous', 'generously', 'gentleness', 'heal', 'healed', 'healer', 'healers', 'healing', 'heals', 'health', 'healthcare', 'healthier', 'healthiness', 'healthy', 'help', 'helped', 'helper', 'helpers', 'helpful', 'helpfulness', 'helping', 'helps', 'hospitable'

Examples of dictionary values:

In [6]:
mfd_dict['care.virtue'][:5]

['alleviate', 'alleviated', 'alleviates', 'alleviating', 'alleviation']

In [7]:
mfd_dict['authority.vice'][:5]

['anarchist', 'anarchistic', 'anarchists', 'anarchy', 'apostate']

## 2 - Translation of the dictionary step by step

### 2.1: DEEPL API

- Rate limits:

We will use the DeepL API to translate all the words in this dictionary to Spanish. There is a cap on 500,000 characters / month. Let us see whether our dictionary would surpass this cap.

In [9]:
total_words = len([word for words in mfd_dict.values() for word in words])
print('There are ' + str(total_words) + ' words in our sample.')

There are 2103 words in our sample.


In [10]:
total_chars = sum([len(word) for words in mfd_dict.values() for word in words])
print('There are ' + str(total_chars) + ' characters in our sample.')

There are 17709 characters in our sample.


Hence, we will be able to translate the whole dictionary without surpassing the rate limits.

- DeepL API setup:

In [46]:
# Generate object with API key and save it.
# auth_key = '63d73ce8-92e8-cb28-67ed-360af519c822:fx'
# with open('api_keys/deepl_api_key.txt', 'w') as f:
#     f.writelines(auth_key)
    
# Load the API key:
with open('api_keys/deepl_api_key.txt', 'r') as f:
    auth_key = f.readline()    
    
# Generate the translator object:
deepl_translator = deepl.Translator(auth_key)

- DeepL API translation:

We will translate each of the words in the MFD and check whether they are correctly translated. To this end, we will create tuples with the English word and its Spanish counterpart.

In [133]:
# Create the dictionary in Spanish
mfd_dict_esp = {}

# The deepl package already allows to instantly translate a list of words,
# way faster than calling the API word by word.
for moral_foundation, words in mfd_dict.items():
    result = deepl_translator.translate_text(words, 
                                             source_lang = "EN",
                                             target_lang = "ES")
    es_words = [word_object.text for word_object in result]
    mfd_dict_esp[moral_foundation] = es_words
        
    print("Moral foundation", moral_foundation, "completed.")

Moral foundation care.virtue completed.
Moral foundation care.vice completed.
Moral foundation fairness.virtue completed.
Moral foundation fairness.vice completed.
Moral foundation loyalty.virtue completed.
Moral foundation loyalty.vice completed.
Moral foundation authority.virtue completed.
Moral foundation authority.vice completed.
Moral foundation sanctity.virtue completed.
Moral foundation sanctity.vice completed.


Comparison of the resulting dictionaries:

In [129]:
mfd_dict_esp["care.virtue"][:5], mfd_dict["care.virtue"][:5]

(['aliviar', 'aliviado', 'alivia', 'Aliviando', 'alivio'],
 ['alleviate', 'alleviated', 'alleviates', 'alleviating', 'alleviation'])

In [1]:
# Store data (serialize)
#with open('dictionaries/deepl_translations.pickle', 'wb') as handle:
#    pickle.dump(mfd_dict_esp, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [2]:
with open('dictionaries/deepl_translations.pickle', 'rb') as handle:
    mfd_dict_esp = pickle.load(handle)

### 2.2: GOOGLE TRANSLATE:

In order to improve the range of translations and capture words that might otherwise go unnoticed, we will also use the Google Translate API through a [Python module](https://github.com/lushan88a/google_trans_new). 


Try another translation method, through Google translate:

In [18]:
#!pip install google_trans_new

In [51]:
# Setup:
trans = google_translator()

In [61]:
# Translation process:
mfd_dict_es_2 = {}
for moral_foundation, words in mfd_dict.items():
    es_words = []
    
    for word in words:
        # Translate word
        es_word = trans.translate(word, lang_src = 'en', lang_tgt = 'es')
        
        # Remove whitespace:
        if type(es_word) == list:
            es_word = [word.strip() for word in es_word]
        else:
            es_word = es_word.strip()
            
        # Append to list of words
        es_words.append(es_word)
            
    # Finally, append the words to the part of the dictionary where it belongs:
    mfd_dict_es_2[moral_foundation] = es_words
    
    # Print ending message:
    print("Moral foundation", moral_foundation, "completed.")

## 3. CHECK TRANSLATIONS:

- We will manually check whether each of the words / expressions makes sense. Try to find alternatives for those that do not work and compare.

In [108]:
# Create a dictionary that shows English - Spanish translations in tuples:
mfd_dict_en_es = {}
for moral_foundation, words in mfd_dict.items():
    mfd_dict_en_es[moral_foundation] = []
    n_words = len(words)
    
    # For each word, get its Spanish translation (they are index equivalent)
    for ith_word in range(n_words):
        en_word = words[ith_word]
        esp_word_deepl = mfd_dict_esp[moral_foundation][ith_word]
        esp_word_google = mfd_dict_es_2[moral_foundation][ith_word]
        mfd_dict_en_es[moral_foundation].append([en_word,
                                                 esp_word_deepl,
                                                 esp_word_google])

- Create an index that shows what words are appropriate for each translation:

This will be a semi-automated process, first I will assume that when both words are the same they will be ok (I will revise them later, but this will remove the burden of needing to read and specify many word pairs). After these have been discarded, I will iterate through the remaining ones and determine which is the best translation.

Codebook:

0) Placeholder for other codes (introduced at the beginning), or it can also show that none of the translations for a word in the original MFD is satisfactory.
1) Get the first translation (DeepL)
2) Get the second translation (Google)
3) Get both translations (if the Google translation is a list we will later transform it using a set)
4) Both translations are the same and they are valid (used in the automated process).

Finally, I will also specify strings with proper comparisons.

In [109]:
# Automated translation, flag those which are the same:
for _, translations in mfd_dict_en_es.items():
    for translation in translations:
        if translation[1] == translation[2]:
            translation.append(4)
        else:
            translation.append(0)

In [121]:
### Manual translation, flag which of them are valid:
for mfd, translations in mfd_dict_en_es.items():
    print(mfd)
    for translation in translations:
        # If 1: Pick only the first one
        # If 2: Pick the first two
        # If 3: Pick both
        # Else (string): New translation
        if translation[3] != 0:
            continue
        else:
            print(translation)
            pick_translation = input('Which is the right translation for this word?')
            translation[3] = pick_translation

care.virtue
care.vice
fairness.virtue
fairness.vice
loyalty.virtue
loyalty.vice
authority.virtue
authority.vice
sanctity.virtue
['bloodiness', 'sanguinolencia', 'crueldad', 0]


Which is the right translation for this word? 1


['bloody', 'maldito', ['sangriento', 'sangrienta'], 0]


Which is the right translation for this word? 2


['buddha', 'buda', 'Buda', 0]


...

Which is the right translation for this word? 3


['whorehouses', 'puticlubs', 'casas', 0]


Which is the right translation for this word? 1


['whoring', 'puta', 'prostituta', 0]


Which is the right translation for this word? 3


- After translating the non-automatic words, there have been several manual erros while coding the proper translations. I will fix them:

Corrections: "molestando", change the first "2[1]",  "1bm", "33", "civil right" = derechos (not "derecho civil"), "cheat" (include "hacer trampa"), "false witness" (incluir ambas), include 3, 'supervises': 3, 'subverting': 3, 'faiths': 3, 'jesus': 2, 'waste': 3, 'wasted': 3
Remove "compañía"?, it can also be interpreted as business company.

In [45]:
### Manual correction of errors
for mfd, translations in mfd_dict_en_es.items():
    print(mfd)
    for translation in translations:
        if (translation[3] in ["2[1]", "1bm", "33"] or
            translation[0] in ["civil right", "cheat", "false witness", "supervises",
                               "subverting", "faiths", "jesus", "waste", "wasted"]):
            print(translation)
            pick_translation = input('Which is the right translation for this word?')
            translation[3] = pick_translation

care.virtue
['empathized', 'empatiza con', ['empatizado', 'empatizada'], '2[1]']


Which is the right translation for this word? 2


care.vice
['unkind', 'poco amable', 'cruel', '1bm']


Which is the right translation for this word? 1


fairness.virtue
['civil right', 'derecho civil', 'derecho Civil', '1']


...

Which is the right translation for this word? 2


sanctity.vice
['waste', 'residuos', 'desperdicio', '1']


Which is the right translation for this word? 3


['wasted', 'desperdiciado', ['desperdiciado', 'desperdiciada'], "['desechado', 'desechada']"]


Which is the right translation for this word? 3


- Check that all words with the same translation are correct 

In [136]:
#### Manual check of similar words
for mfd, translations in mfd_dict_en_es.items():
    print(mfd)
    for translation in translations:
        # Code: Same as before
        if translation[3] == 4:
            print(translation)
            pick_translation = input('Which is the right translation for this word?')
            translation[3] = pick_translation

care.virtue
care.vice
fairness.virtue
fairness.vice
loyalty.virtue
loyalty.vice
authority.virtue
authority.vice
sanctity.virtue
sanctity.vice


Manual errors correction:

'console': ['consolar', 'consuelo'], 'abuses': 'abusa', 'aching': ['dolor', 'doliendo'], 'threatening': ['amenazante', 'amenazador'], 'level the playing field': ['nivelar', 'nivela', 'nivelado', 'igualdad de condiciones'], 'trusted': ['de confianza', 'confianza', 'confiar'], 'be partial': ['parcial', 'parcialidad', 'ser parcial'], 'prejudice': ['prejuicio', 'prejuzgar'], 'scamming': 'estafando', 'company'?, 'herding': 'en manada', 'together': ['juntos', 'juntas', 'unidos', 'unidas'], 'unity': ['unidad', 'unión'], 'backstabbed': ['por la espalda', 'apuñalado por la espalda'], 'monarch': ['rey', 'reinado', 'reinar'], 'punitive': ['punitivo', 'punitiva'], 'refuses': ['se niega', 'negarse'], 'faithfully': 'religiosamente', 'priest': ['sacerdote', 'párroco'], 'repenting': 4, 'repent': ['arrepentirse', 'arrepentimiento'], 'theology': ['teología', 'teológico'], 'foul': ['nauseabundo', 'fétido', 'hediondo', 'repugnante', 'asqueroso', 'inmundo', 'infecto']

In [46]:
### Manual errors in categorizing words correction:
corrections = {'console': ['consolar', 'consuelo'], 'abuses': 
               'abusa', 'aching': ['dolor', 'doliendo'], 
               'threatening': ['amenazante', 'amenazador'], 
               'level the playing field': ['nivelar', 'nivela', 'nivelado', 'igualdad de condiciones'], 
               'trusted': ['de confianza', 'confianza', 'confiar'], 
               'be partial': ['parcial', 'parcialidad', 'ser parcial'], 
               'prejudice': ['prejuicio', 'prejuzgar'], 
               'scamming': 'estafando', 'company': [], 'herding': 'en manada', 
               'together': ['juntos', 'juntas', 'unidos', 'unidas'], 
               'unity': ['unidad', 'unión'], 'backstabbed': ['por la espalda', 'apuñalado por la espalda'],
               'monarch': ['rey', 'reinado', 'reinar'], 'punitive': ['punitivo', 'punitiva'], 
               'refuses': ['se niega', 'negarse'], 'faithfully': 'religiosamente', 
               'priest': ['sacerdote', 'párroco'], 'repenting': 4, 
               'repent': ['arrepentirse', 'arrepentimiento'], 'theology': ['teología', 'teológico'],
               'foul': ['nauseabundo', 'fétido', 'hediondo', 'repugnante', 'asqueroso', 'inmundo', 'infecto']}

#### Manual check of similar words
for mfd, translations in mfd_dict_en_es.items():
    print(mfd)
    for translation in translations:
        # Code: Same as before
        if translation[0] in corrections.keys():
            if translation[0] == 'company':
                print(translation)
                pick_translation = input('Which is the correct translation for this word?')
                translation[3] = pick_translation
                continue
            
            print(translation)
            pick_translation = corrections[translation[0]]
            translation[3] = pick_translation

care.virtue
['console', 'consola', 'consola', 4]
care.vice
['abuses', 'abusos', 'abusos', 4]
['aching', 'dolor', 'dolor', 4]
['threatening', 'amenazante', 'amenazante', 4]
fairness.virtue
['level the playing field', 'nivelar el campo de juego', 'nivelar el campo de juego', 4]
['trusted', 'de confianza', 'de confianza', 4]
fairness.vice
['be partial', 'ser parcial', 'ser parcial', 4]
['prejudice', 'prejuicio', 'prejuicio', 4]
['scamming', 'estafa', 'estafa', 4]
loyalty.virtue
['company', 'empresa', 'empresa', 4]


Which is the correct translation for this word? 4


['herding', 'pastoreo', 'pastoreo', 4]
['together', 'juntos', 'juntos', 4]
['unity', 'unidad', 'unidad', 4]
loyalty.vice
['backstabbed', 'apuñalado por la espalda', 'apuñalado por la espalda', 4]
authority.virtue
['monarch', 'monarca', 'monarca', 4]
['punitive', 'punitivo', 'punitivo', 4]
authority.vice
['refuses', 'se niega', 'se niega', 4]
sanctity.virtue
['faithfully', 'fielmente', 'fielmente', 4]
['priest', 'sacerdote', 'sacerdote', 4]
['repent', 'arrepentirse', 'arrepentirse', 4]
['repenting', 'arrepentido', 'arrepentido', 4]
['theology', 'teología', 'teología', 4]
sanctity.vice
['foul', 'falta', 'falta', 4]


- Final words to include from those we thought were bad translations:

In [47]:
corrections = {'torturous': "2", "bigoted": "1", 
               "duplicitous": ["artero", "artera", "arteros", "arteras",
                               "taimado", "taimada", "taimados", "taimadas"],
               
               "false advertised": ["falso", "falsa", "falsos", "falsas"]
               
              }

for foundation, translations in mfd_dict_en_es.items():
    for translation in translations:
        if translation[3] == "0" and translation[0] in corrections.keys():
            translation[3] = corrections[translation[0]]

In [48]:
# Store data (serialize)
with open('dictionaries/mfd_en_esp_manual.pickle', 'wb') as handle:
    pickle.dump(mfd_dict_en_es, handle, protocol = pickle.HIGHEST_PROTOCOL)

## 4. EXTRACT TRANSLATIONS AND AVOID REPETITIONS

Here we will use the previous codebook to know which translation to pick between DeepL, Google Translate, and our own personal translations.

In [80]:
with open('dictionaries/mfd_en_esp_manual.pickle', 'rb') as handle:
    mfd_dict_en_es = pickle.load(handle)

In [81]:
# Iterate through the dictionary and use the previous codebook to insert words in a list.
mfd_dict_es_final = {}
for foundation, translations in mfd_dict_en_es.items():
    print(foundation)
    mfd_dict_es_final[foundation] = []
    for translation in translations:
        # Set both translations in lowercase
        translation[1] = translation[1].lower()
        type_google_translate = type(translation[2])
        if type_google_translate == list:
            for word in range(len(translation[2])):
                lowercase_word = translation[2][word].lower()
                translation[2][word] = lowercase_word
        else:
            translation[2] = translation[2].lower()
            
        # Get the number of the translation
        input_translation = str(translation[3])
        
        # If it is a number, pick the translation following the codebook
        if re.search("[0-9]", input_translation):
            # 3 = pick both translations
            if input_translation == "3":
                # We only need to check that the second one is a list
                # Add only new words
                if translation[1] not in mfd_dict_es_final[foundation]:
                    mfd_dict_es_final[foundation].append(translation[1])
                if type_google_translate == list:
                    for esp_word in translation[2]:
                        if esp_word not in mfd_dict_es_final[foundation]:
                            mfd_dict_es_final[foundation].append(esp_word)
                else:
                    if translation[2] not in mfd_dict_es_final[foundation]:
                        mfd_dict_es_final[foundation].append(translation[2])
            # 1 or 4, pick first translation
            elif input_translation in ["1", "4"]:
                if translation[1] not in mfd_dict_es_final[foundation]:
                    mfd_dict_es_final[foundation].append(translation[1])
            elif input_translation == "2":
                if type_google_translate == list:
                    for esp_word in translation[2]:
                        if esp_word not in mfd_dict_es_final[foundation]:
                            mfd_dict_es_final[foundation].append(esp_word)
                else:
                    if translation[2] not in mfd_dict_es_final[foundation]:
                        mfd_dict_es_final[foundation].append(translation[2])
                        
            # If there is a zero we will not consider it
            
            
        # If the string is not a number, but a word or list of words that we inputted,
        # then we add them to our list of translations
        else:
            if input_translation != "":
                # Pattern to match words in the shape of a list as we inputted them while translating
                # It takes all words separated by a whitespace (so that n-grams are included)
                pattern = "\s*\w+(?:\s+\w+)*\s*"
                translation_ls = re.findall(pattern, input_translation)
                n_words = len(translation_ls)
                if n_words > 1:
                    for esp_word in translation_ls:
                        if esp_word not in mfd_dict_es_final[foundation]:
                            mfd_dict_es_final[foundation].append(esp_word)
                elif n_words == 1:
                    if translation_ls[0] not in mfd_dict_es_final[foundation]:
                        mfd_dict_es_final[foundation].append(translation_ls[0])
            
    # Finally, get the unique words using a set and transform it back into a list.
    #unique_words = set(mfd_dict_es_final[foundation])
    #mfd_dict_es_final[foundation] = list(unique_words)

care.virtue
care.vice
fairness.virtue
fairness.vice
loyalty.virtue
loyalty.vice
authority.virtue
authority.vice
sanctity.virtue
sanctity.vice


In [82]:
for foundation, words in mfd_dict_es_final.items():
    print(foundation, ":", words[:5])

care.virtue : ['aliviar', 'aliviado', 'aliviada', 'alivia', 'aliviando']
care.vice : ['abusado', 'abusada', 'abusador', 'abusadora', 'abusadores']
fairness.virtue : ['vengarse', 'vengar', 'vengado', 'vengada', 'vengador']
fairness.vice : ['soy parcial', 'embaucar', 'engatusar', 'embaucado', 'engañado']
loyalty.virtue : ['todos para uno', 'lealtad', 'lealtades', 'leal', 'aliado']
loyalty.vice : ['contra nosotros', 'apóstata', 'apóstatas', 'apuñalar por la espalda', 'por la espalda']
authority.virtue : ['consentir', 'ceder', 'consintió', 'conformado', 'conformada']
authority.vice : ['anarquista', 'anarquistas', 'anarquía', 'apóstata', 'apóstatas']
sanctity.virtue : ['abstinencia', 'alá', 'todopoderoso', 'todopoderosa', 'ángel']
sanctity.vice : ['aborrecer', 'aborrecido', 'aborrecida', 'aborrece', 'adicto']


- Let us store this raw dictionary. Later it can be modified by:
    1) stemming
    2) including / removing certain context-specific words with semantic importance. 

In [83]:
#with open('dictionaries/raw_mfd_esp.pickle', 'wb') as handle:
#    pickle.dump(mfd_dict_es_final, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [6]:
with open('dictionaries/raw_mfd_esp.pickle', 'rb') as handle:
    mfd_dict_es_final = pickle.load(handle)

## 5. CHECK THE INCLUSION AND ELIMINATION OF ADDITIONAL CONTEXT-SPECIFIC WORDS

- Definitions of loyalty and authority according to The Righteous Mind, by Jonathan Haidt:

> Loyalty: The Loyalty/betrayal foundation is just a part of our innate preparation for meeting the adaptive challenge of forming cohesive coalitions. The original trigger for the Loyalty foundation is anything that tells you who is a team player and who is a traitor, particularly when your team is fighting with other teams.

> Authority: "The current triggers of the Authority/subversion foundation, therefore, include anything that is construed as an act of obedience, disobedience, respect, disrespect, submission, or rebellion, with regard to authorities perceived to be legitimate."

- There are several words that represent certain moral foundations for the particular case of Spain, these are: 

1) Mentioning the name of the country is usually associated with glorifying the concept of nation:
    - `Loyalty.virtue: ['españa', 'españoles', 'españolas', 'español', 'española', 'unidad de españa']`

2) The independence process in Catalonia is considered a threat to the group (could be either against loyalty or against authority). Some politicians mention it for the purposes of exalting their respect for the nation. I need references here on whether it is **loyalty or authority**.
    - `Loyalty.vice: ['separatista', 'separatistas', 'separatismo', 'secesión', 'secesionista', 'secesionistas', 'secesionismo', 'independentismo', 'independentista', 'independentistas']`

3) Particularly, the far right party also refers to the attempt of the Catalonian parliament to claim independence as a coup, and there are many related words to this:
    - `authority.vice: ['golpista', 'golpistas', 'golpe de estado']`
    - (Also 'eta' and 'etarra' have been added to the final dictionary as violations of the loyalty foundation, although after the stemming process)

In [86]:
extended_mfd_esp = deepcopy(mfd_dict_es_final)
new_words_dict = {'loyalty.virtue': ['españa', 'españoles', 'españolas', 'español',
                                     'española', 'unidad de españa'],
                  'loyalty.vice': ['separatista', 'separatistas', 'separatismo', 
                                   'secesión', 'secesionista', 'secesionistas', 
                                   'secesionismo', 'independentismo', 
                                   'independentista', 'independentistas'],
                  'authority.vice': ['golpista', 'golpistas', 'golpe de estado']}

for foundation, words in new_words_dict.items():
    extended_mfd_esp[foundation].extend(words)

- Words that might pose problems and could be removed after some sanity checks:

1) 'Comunidad', 'comunidades'. These are the name of the regional administrations of Spain (e.g., "Comunidad de Madrid"), and we should take that into account (otherwise it will be difficult to filter out the semantics of mentioning the sense of community against power problems between administrations in Spain).
2) 'compañía', 'compañías'. If it is understood as accompanying somebody it is ok, but the far left party often uses it because it also means "company" in the business sense. 
3) 'Beneficio', 'beneficios'. These could be good if we understand them as the benefits of a food for healing, or how your presence benefits me. Nevertheless, they are also usually used by the far left to say "profit / profits".
4) 'Clasificado': This does not only mean ranked, but also classified (less important).
5) 'Iglesias': Could mean churches, but it is also the surname of the far left political candidate. Could be erased if it is present in many texts. 
6) 'Fiel': It is the translation of faithful in a religious sense, but also for the moral foundation of loyalty.virtue.

- Let us search those words in the tweets dataset:

In [87]:
tweets_df = pd.read_csv("../data/tweets_df.csv")

1) Comunidad / Comunidades

In [88]:
tweets_df[tweets_df.text.str.contains("comunidad", case = False)].loc[:, ["screen_name", "text"]]

,screen_name,text
162,vox_es,"📰 @Santi_ABASCAL proclama en Valencia que la Comunidad Valenciana ""es la vanguardia nacional al padecer la vecindad del separatismo"".\n\nVOX ha logrado reunir a 8.000 personas en el acto más multitudinario de la campaña valenciana.\n\n⬇️⬇️⬇️\n https://t.co/zrb6DKBXZC"
278,vox_es,"@monasterioR @telemadrid 🗨 ""Cuanto más trabajo dentro de en la Comunidad de Madrid, más segura estoy de que hay que desmontar las Autonomías"".\n\n📢 #MonasterioTelemadrid https://t.co/pZCM3z5FR6"
309,vox_es,"@ivanedlm @EspejoPublico 🗨 @ivanedlm ""Rivera ha disparado el número de altos cargos en la Comunidad de Madrid nada más llegar al gobierno para dar la cabida suficiente a todos sus enchufados"". https://t.co/gOhGGqYszv"
622,populares,"▶️ @pablocasado_: ""Las Comunidades Autónomas han acercado la administración a todos los españoles. Nuestros gobiernos son austeros, bajan impuestos y lideran la sanidad y educación. El problema no son las autonomías, sino los desleales"". https://t.co/gwRPxHGBmh"
658,populares,"📺 @anapastorjulian: ""Las Comunidades Autónomas han acercado la gestión de los servicios públicos a los ciudadanos. Tenemos que garantizar la cohesión y la calidad en el Sistema Nacional de Salud; con las mismas prestaciones en toda España"".\n\n#ÚltimaOportunidadL6 https://t.co/tpzzZFrMeL"
688,populares,"""Mi compromiso es firme: cuando sea presidente del gobierno voy a mejorar la financiación autonómica de todos los valencianos, es de justicia. No puede haber un agravio comparativo así"".\n\n📡 @pablocasado_ se compromete con la Comunidad Valenciana\n#PorTodoLoQueNosUne https://t.co/MSFKrrFieQ"
690,populares,"⏩ @isabelbonig: ""Necesitamos un gobierno que vuelva a instalar la libertad educativa en las aulas. \nEsta Comunidad siente en castellano como en valenciano"".\n\n#PorTodoLoQueNosUne https://t.co/pBd8Xy6zhF"
692,populares,"⏩ @isabelbonig: ""Esta tierra te necesita, @pablocasado_, porque su compromiso con esta Comunidad es patente y ha hecho que te hayas ganado la admiración de todo el pueblo valenciano. Por ello este pueblo el #10N te devolverá su compromiso, te lo garantizo"".\n\n#PorTodoLoQueNosUne https://t.co/3uWEURpeca"
694,populares,"⏩ @BelenHoyo: ""Un buen político tiene que mirar hacia delante; frenando la desaceleración económica, creando empleo o bajando los impuestos. Necesitamos impulsar nuestra industria, llevando a cabo políticas responsables"".\n\n#PorTodoLoQueNosUne en la Comunidad Valenciana https://t.co/AOHBbeQiqe"
696,populares,"⏩@mjosecatala: ""Los valencianos no necesitamos un presidente que esté mirando al suelo. Queremos un presidente que nos mire a los ojos y nosdiga que va a estar con esta comunidad en materia de infraestructuras o financiación"" \n👉Es lo que hace @pablocasado_.\n\n#PorTodoLoQueNosUne https://t.co/WqFuSLBz0O"


Out of the 35 tweets in which the word "comunidad" or "comunidades" appear, in 34 of them they have an administrative instead of group meaning. Recommended deletion.

2) Compañía(s)

In [89]:
tweets_df[tweets_df.text.str.contains("compañía", case = False)].loc[:, ["screen_name", "text"]]

,screen_name,text
60,Santi_ABASCAL,"De lo mejor de echarse a la carretera para hacer campaña electoral es la compañía y el ejemplo de este hombre bueno y heroico, José Antonio Ortega Lara. #Oviedo #EspañaSiempre 🇪🇸 https://t.co/eweHGV2ltM"
981,populares,"De visita por Toledo en la inmejorable compañía de @marianorajoy, @paconunez_ y tantos amigos del @populares @PP_CLM. Seguimos siendo el partido que hace crecer a España siempre que gobierna. El #10N tenemos la oportunidad de volver a lograrlo. #PorTodoLoQueNosUne #VotaPP https://t.co/fIyvtRGlXh"
1564,CiudadanosCs,"De camino a Castrejón de Trabancos a visitar la cooperativa COCETRA con @owoniqui. Con compañía así, al fin del mundo!\n@Valladolid_Cs @CsCastillayLeon https://t.co/kk3JktwoBc"


Surprisingly, here the word is used with a companionship meaning.

3) Beneficio(s)

In [90]:
tweets_df[tweets_df.text.str.contains("beneficio", case = False)].loc[:, ["screen_name", "text"]]

,screen_name,text
312,vox_es,"#ESPAÑAVOTA | LA ENTREVISTA \n@monasterioR : ""La progresividad es importante, pero no se puede desincentivar tener beneficios"" \n📡 directo https://t.co/ygpf7Gbuwy\n📲 #MonasterioTelemadrid \n@vox_es @madrid_vox \n \n@madrid_vox https://t.co/ym7vS8IYrL"
651,populares,📺 @gmariscalanaya en el #debatee10Ncongresortve: El aumento de gasto trae beneficio a la sociedad. Con menos impuestos mas capacidad de gasto de los canarios y mayor crecimiento de la economía. https://t.co/SMofU0APui
673,populares,"🎙 @cucagamarra en @RadioRioja 👉 ""Nuestro proyecto político está abierto a todo el votante de centro-derecha. Los españoles saben que sabemos gestionar bien con políticas eficaces en beneficio de todos"".\n\nEl interés general es lo que nos mueve para trabajar por España."
1121,Albert_Rivera,- ¿En qué ha trabajado usted?\n\n- Adriana Lastra: “Pues...ehm... en muchas cosas... que no tengo por qué explicitar porque yo no hablo de mi vida privada”.\n\nOtra enchufada sin oficio ni beneficio capaz de llegar a ser diputada y portavoz del @PSOE. https://t.co/ADTUxSbzOF
1212,CiudadanosCs,Para que nuestros #agricultores y #ganaderos puedan garantizar la rentabilidad de sus explotaciones mediante la mejora de la Ley de la Cadena Alimentaria y que los precios cubran los costes y los beneficios. #VotaCiudadanos https://t.co/I1vANgPxTP
1238,CiudadanosCs,"Los amigos y familiares de @XimoPuig ya sabían lo que ocurriría cuando los socialistas ganaran:\n\n«¡Nos vamos a forrar!»\n\nY acertaron: facturas duplicadas, lluvia de subvenciones, programas fantasma, +300% en beneficios...\n\nSe llama clientelismo.\nSe llama socialismo. https://t.co/nqsOdNaoE1"
1576,CiudadanosCs,Nuestra propuesta:\n✅ Tarifa plana de autónomos 2 años\n✅ Bonificación del 100% al 1er contrato de jóvenes\n✅ Impuesto de Sociedades del 15% para PYMES que reinviertan beneficios\n✅ Eliminar contratos basura: todos indefinidos\n✅ Las Startups no pagarán impuestos en 2 años
1672,sanchezcastejon,"Pido el voto al @PSOE para tener un Gobierno fuerte, estable, que nos permita construir convivencia en España, hacer políticas sociales en beneficio de los españoles y españolas. La cohesión social y territorial deben ir de la mano, nos hacen mejores como sociedad. #ElDebate4N https://t.co/O28yFoRbvJ"
1778,PSOE,🗣El #PSOE siempre defenderá la cohesión social y territorial porque nos hacen mejores como país y sociedad\n\nPido el voto para tener un Gobierno fuerte y estable que permita construir convivencia en #España y políticas sociales en beneficio de todos los españoles\n\n#VotaConCorazón https://t.co/buLoCWxjKN
1958,PSOE,"La cohesión social y territorial nos hacen mejor como país y como sociedad. \n\nSi quieres un 𝗚𝗼𝗯𝗶𝗲𝗿𝗻𝗼 𝗳𝘂𝗲𝗿𝘁𝗲 𝘆 𝗲𝘀𝘁𝗮𝗯𝗹𝗲 que permita construir convivencia en España y permita hacer políticas sociales en beneficio de la ciudadanía, #VotaPSOE /❤ \n\n#AhoraSí https://t.co/ZrNHKsvIir"


Out of 10 tweets that include this word, 5 times it is used meaning profit, whereas in the other 5 it means improvement of the population. Doubt

4) Clasificado/a

In [91]:
tweets_df[tweets_df.text.str.contains("clasificad*", case = False)].loc[:, ["screen_name", "text"]]

,screen_name,text


Does not seem like a concern.

5) Iglesias:

In [92]:
tweets_df[tweets_df.text.str.contains("iglesias", case = False)].loc[:, ["screen_name", "text"]]

,screen_name,text
46,Santi_ABASCAL,"Santiago Abascal pulveriza a Pablo Iglesias: ""Yo me jugaba la vida amenazado por ETA en el País Vasco mientras usted iba a las herriko tabernas"" https://t.co/l4ipLDzvmo https://t.co/6lIE4ocmCk"
69,Santi_ABASCAL,📺 La audiencia de @El_Hormiguero:\n\n@Santi_ABASCAL - 4.049.000\nRivera - 2.458.000\nErrejón - 2.089.000\nCasado - 2.049.000\nIglesias - 1.477.000\n\nY eso que los únicos boicoteados éramos nosotros. Millones de españoles ya no obedecen a la dictadura progre 💪🏻
321,vox_es,"Santiago Abascal pulveriza a Pablo Iglesias: ""Yo me jugaba la vida amenazado por ETA en el País Vasco mientras usted iba a las herriko tabernas"" https://t.co/l4ipLDzvmo https://t.co/6lIE4ocmCk"
325,vox_es,⚡ Monumental repaso de Santiago Abascal a Pedro Sánchez y Pablo Iglesias por la memoria histórica https://t.co/zzhFzaoyBn vía @ldtv
345,vox_es,"""No hay que elegir entre nuestros abuelos"" \n""Nuestros abuelos se abrazaron, nuestros padres convivieron y usted quiere enfrentar a los nietos"" \n@Santi_ABASCAL a Sánchez. \n(Y de paso, un sopapo a Iglesias justo al principio por Comunista)\n\n#debate5rtve https://t.co/deoprsuPoo"
347,vox_es,"Abascal a Sánchez: ""Si Vd. quiere ilegalizar los totalitarismos, quizá acabará ilegalizando a su socio Iglesias, que es el único de este debate que simpatiza con una ideología de este corte, en su caso el comunismo""\n#DebateElectoral"
348,vox_es,".@Santi_ABASCAL a Pablo Iglesias: ""Soy el único que se ha jugado la vida mientras usted estaba en una herriko taberna y Pedro Sánchez jugaba al baloncesto"".\n\n#ElDebate4N"
349,vox_es,"📺 @Santi_ABASCAL pulveriza a Pablo Iglesias ""No me va a dar lecciones de democracia ni de defensa del orden constitucional. Yo me jugaba la vida amenazado por ETA en el País Vasco mientras usted presumía de que ETA tenía perspicacia política"".\n\n📢 #ElDebate4N #DebateElectoral https://t.co/GtRi8aT0EL"
361,vox_es,"Y @Santi_ABASCAL siguiendo la tradición de Iván y Ortega Smith, los pone en fila y les va sacudiendo uno a uno, Sánchez, Casado, Iglesias... Que no pueden hacer otra cosa que mirar al suelo y poner cara de que pase rápido. \n#debatea5RTVE https://t.co/Io4IKYyRVw"
467,vox_es,📺 La audiencia de @El_Hormiguero:\n\n@Santi_ABASCAL - 4.049.000\nRivera - 2.458.000\nErrejón - 2.089.000\nCasado - 2.049.000\nIglesias - 1.477.000\n\nY eso que los únicos boicoteados éramos nosotros. Millones de españoles ya no obedecen a la dictadura progre 💪🏻


Almost all tweets refer to the surname of the politician and not the concept of churches. Furthermore, it is quite common to use it. It should be deleted.

Let us check that "Iglesia" (the singular, which would mean church), does not appear as a misspelling of the politician surname.

In [93]:
tweets_df[tweets_df.text.str.contains("\\biglesia\\b", case = False)].loc[:, ["screen_name", "text"]]

,screen_name,text


Iglesia does not appear in any tweet, so we can leave it.

6) Fiel (sanctity.virtue):

In [94]:
tweets_df[tweets_df.text.str.contains("fiel", case = False)].loc[:, ["screen_name", "text"]]

,screen_name,text
125,vox_es,"📺 @sanchezdelreal en #ExtremaduraDebate\n""En una comisión el Ministro Ábalos prometía decenas de miles de millones a los separatistas, mientras el diputado del PSOE por Badajoz no pestañeó. Los extremeños han sido fieles pero no van a ser tontos""\n\n⬇Hilo con sus intervenciones⬇ https://t.co/Ol796ho1DU"
1143,CiudadanosCs,🇪🇸 ¡Necesitamos al frente del Gobierno a un hombre como @Albert_Rivera!\n\n💪 Él sabe lo que es dejarse la piel por todos los españoles.\n🧡 ¡Su valentía es el fiel reflejo de la de sus padres!\n\n▶️@InesArrimadas en el cierre de campaña en #Barcelona 👏 #YoVotoCs https://t.co/b3FDvpTc6I
1158,CiudadanosCs,"🔝 @InesArrimadas ""Necesitamos al frente del gobierno de España a un hombre como @Albert_Rivera, que sabe lo que es dejarse la piel por España"" #YoVotoCs🍊\n♥️💪 Su valentía es fiel reflejo de la de sus padres. ¡Gracias, Agustín y María Jesús! https://t.co/9Kg25NSuWY"


Even though "fiel" is hardly ever used, it does not have the sanctity meaning. Its `sanctity.vice` meaning should be deleted.

In [95]:
# Removing words:
words_to_remove = {'loyalty.virtue': ['comunidad', 'comunidades'], 
                   'sanctity.virtue': ['iglesias', 'fiel']}

for foundation, words in words_to_remove.items():
    for word in words:
        print(word, "has been removed.")
        extended_mfd_esp[foundation].remove(word)

comunidad has been removed.
comunidades has been removed.
iglesias has been removed.
fiel has been removed.


In [96]:
#with open('dictionaries/extended_mfd_esp.pickle', 'wb') as handle:
#    pickle.dump(extended_mfd_esp, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [97]:
with open('dictionaries/extended_mfd_esp.pickle', 'rb') as handle:
    extended_mfd_esp = pickle.load(handle)

## 6. STEM WORDS

Given that in Spanish there are many word terminations (like verb tenses, plural, masculine and femenine terminations), stemming sounds like a reasonable option. Let us try a Spanish stemmer and manually check whether the resulting dictionary would have many false positives.

In [189]:
stemmer_esp = SnowballStemmer("spanish")
snowball_stem_comparisons = {}
for foundation, words in extended_mfd_esp.items():
    print(foundation)
    snowball_stem_comparisons[foundation] = []
    for word in words:
        stem_word = stemmer_esp.stem(word)
        #if stem_word not in snowball_stem_comparisons[foundation]:
        snowball_stem_comparisons[foundation].append(stem_word)

care.virtue
care.vice
fairness.virtue
fairness.vice
loyalty.virtue
loyalty.vice
authority.virtue
authority.vice
sanctity.virtue
sanctity.vice


In [190]:
# Get the smallest stem that is contained in a string for each foundation
for foundation, words in snowball_stem_comparisons.items():
    print(foundation)
    for word_stem in range(len(words)):
        for other_word_stem in words:
            if other_word_stem in words[word_stem] and other_word_stem != words[word_stem]:
                words[word_stem] = other_word_stem

care.virtue
care.vice
fairness.virtue
fairness.vice
loyalty.virtue
loyalty.vice
authority.virtue
authority.vice
sanctity.virtue
sanctity.vice


In [191]:
# Let us create a table with the stemming and export it to excel:
stems_df = pd.DataFrame(snowball_stem_comparisons.items()).explode(1)
originals_df = pd.DataFrame(extended_mfd_esp.items()).explode(1)
stemming_df = pd.concat([originals_df.rename({0: 'foundation', 1: 'original_word'}, axis = 1), 
                         stems_df.rename({1: 'stem'}, axis = 1).drop(0, axis = 1)], axis = 1).reset_index(drop = True)

We will use an Excel file to choose which stem makes most sense:

In [192]:
stemming_df.to_excel("../data/raw/stems.xlsx", index = False)

Here we will manually correct words that might capture other meanings.

care.virtue: Embrutecer should not have the beginning of the word trimmed. (I.e., we are using "brut", but it should capture "embrutecer").
care.vice: The same for puñal. (It should also include "puñala")

### After the manual review of the stems, we read the resulting Excel and select unique stems to end our Spanish MFD dictionary:

In [30]:
clean_stems_df = pd.read_excel("../data/clean/stems_clean.xlsx")
clean_stems_df

,foundation,original_word,stem,possible_double_meaning,Notes,Doubt
0,care.virtue,aliviar,alivi,NaN,NaN,NaN
1,care.virtue,aliviado,alivi,NaN,NaN,NaN
2,care.virtue,aliviada,alivi,NaN,NaN,NaN
3,care.virtue,alivia,alivi,NaN,NaN,NaN
4,care.virtue,aliviando,alivi,NaN,NaN,NaN
...,...,...,...,...,...,...
2340,sanctity.vice,desechos,desech,NaN,NaN,NaN
2341,sanctity.vice,desperdiciando,desperdici,NaN,NaN,NaN
2342,sanctity.vice,prostíbulo,prost[ií],NaN,NaN,NaN
2343,sanctity.vice,burdel,burdel,NaN,NaN,NaN


In [32]:
# This code allows to see whether there are smaller stems than the ones we
# are looking for (which would create duplicated counts for the same word in the same foundation):
final_esp_mfd = {foundation: [] for foundation in mfd_dict_es_final.keys()}
for row in range(clean_stems_df.shape[0]):
    # Extract stem, original word and foundation
    # Add \b to the beginning of the stem, so that only the term is detected
    # E.g.: "\brice" is not detected in "price", but "rice" is
    # When reading the stem, the \b of the middle of the string is transformed
    # into a raw string (\\b)
    # I would also have added \b at the beginning of each string, but that would
    # hinder legibility.
    stem = clean_stems_df.loc[row, "stem"]
    stem = r'\b' + stem
    
    original_word = clean_stems_df.loc[row, "original_word"]
    foundation = clean_stems_df.loc[row, "foundation"]
    final_esp_mfd[foundation].append(stem)

for foundation, words in final_esp_mfd.items():
    print("\n", foundation, "\n")
    for word_id in range(len(words)):
        for other_word in words:
            if other_word != words[word_id] and other_word in words[word_id]:
                print(words[word_id], other_word)  


 care.virtue 


 care.vice 


 fairness.virtue 


 fairness.vice 


 loyalty.virtue 


 loyalty.vice 


 authority.virtue 


 authority.vice 


 sanctity.virtue 


 sanctity.vice 



In [33]:
# Finally, create a dictionary of unique words:
final_esp_mfd = {foundation: [] for foundation in mfd_dict_es_final.keys()}
for row in range(clean_stems_df.shape[0]):
    # Extract stem, original word and foundation
    # Add \b to the beginning of the stem, so that only the term is detected
    # E.g.: "\brice" is not detected in "price", but "rice" is
    # This also allows to capture the term if it is a hashtag before
    stem = clean_stems_df.loc[row, "stem"]
    stem = r'\b' + stem
    
    original_word = clean_stems_df.loc[row, "original_word"]
    foundation = clean_stems_df.loc[row, "foundation"]
    # Append only new stems
    if stem not in final_esp_mfd[foundation]:
        final_esp_mfd[foundation].append(stem)

**Here is our final moral foundations dictionary in Spanish:**

In [34]:
for foundation, words in final_esp_mfd.items():
    print(f'{foundation}: {words[:5]}')

care.virtue: ['\\balivi', '\\baltruis', '\\bbenefice', '\\bbenevolen', '\\bcuid']
care.vice: ['\\babus', '\\bdol[iío]', '\\bafli', '\\bagon', '\\bangusti']
fairness.virtue: ['\\bvenga[rd]|\\bvenganz', '\\bobjetiva|\\bobjetivid', '\\bderecho', '\\bcompens', '\\blegal']
fairness.vice: ['\\bbuitre', '\\bespecul', '\\bpropio?\\s?benefici', '\\boligop', '\\boligar[cq]']
loyalty.virtue: ['\\btodos\\spara\\sun', '\\bleal', '\\baliad', '\\bpertene', '\\breverenc']
loyalty.vice: ['\\bcontra\\snosotr', '\\bapóstat', '\\bpor\\W*la\\W*espalda', '\\btraici[oó]n|traidor', '\\bdesun']
authority.virtue: ['\\bconsi?e?nt', '\\b(?<!los\\W)ceder|ced[aiío]', '\\bconformad', '\\baquiesc', '\\badhi?e?r']
authority.vice: ['\\ban[aá]rq', '\\bapóstat', '\\balgarabía', '\\bcaos|caótic', '\\bdesord']
sanctity.virtue: ['\\babstinen', '\\balá', '\\btodopoder', '\\b[aá]ng[eé]l', '\\bap[oó]st[oó]l']
sanctity.vice: ['\\baborrec', '\\badic[ct]', '\\badúlter|\\badulteri', '\\balcoh[oó]l', '\\bapóstat']


In [35]:
# Store the dictionary
with open("dictionaries/final_mfd_dict_stems.pickle", "wb") as handle:
    pickle.dump(final_esp_mfd, handle, protocol = pickle.HIGHEST_PROTOCOL)

## 7. VALIDATE DICTIONARY